# Forcasting errors
https://arxiv.org/pdf/1006.0609.pdf

In [1]:
import numpy as np 
import pyccl as ccl
from scipy.interpolate import interp1d
from scipy import integrate

In [2]:
c = 299792458.0
z = 0.1
k_arr = np.linspace(0.001, 1, 100)
params = {'h': 0.6727,
  'Omega_cdm': 0.265621, #0.237153,
  'Omega_b': 0.0494116,
  'Omega_Lambda': 0.6834,
  'sigma8': 0.812}
cosmo = ccl.boltzmann.classy.Class()
cosmo.set({'output':'mPk', 'P_k_max_h/Mpc': 20, 'z_max_pk': 1085})
cosmo.set(params)
cosmo.compute()

In [3]:
class Fisher():
    def __init__(self, V0, n_gal):
        self.V0 = V0
        self.n_gal = n_gal
        self.sigmaz = 0.001
        self.c = 299792458.0
        self.z = 0.1
        self.z_arr = np.linspace(0.1, 2, 20)
        self.k_arr = np.linspace(0.001, 1, 100)
        self.params = {'h': 0.6727,
          'Omega_cdm': 0.265621, #0.237153,
          'Omega_b': 0.0494116,
          'Omega_Lambda': 0.6834,
          'sigma8': 0.812}
        self.cosmo = ccl.boltzmann.classy.Class()
        self.cosmo.set({'output':'mPk', 'P_k_max_h/Mpc': 20, 'z_max_pk': 1085})
        self.cosmo.set(self.params)
        self.cosmo.compute()
        self.dchidz = self.get_dchidz()

    def get_fisher(self):
        parts = {'b': self.get_dlnPdb,
                 'f': self.get_dlnPdf,
                'a_par': self.get_dlnPda_par,
                'a_per': self.get_dlnPda_per}
        fisher_mat = {}
        for i, part1 in enumerate(parts.keys()):
            for j, part2 in enumerate(parts.keys()):
                if i <= j:
                    print(part1, part2)
                    fish_submat = {}
                    for ii, z_i in enumerate(self.z_arr):
                        for jj, z_j in enumerate(self.z_arr):
                            print('z_i =', z_i, 'z_j =', z_j)
                            if ii <= jj:
                                fisher_mat[part1+part2] = self.get_fisher_ij(parts[part1],
                                                                             parts[part2],
                                                                             z_i, z_j)
        return fisher_mat

    def get_fisher_ij(self, part1, part2, z_i, z_j):
        #print(self.k_arr[0], self.k_arr[-1])
        integrand = self.get_fisher_ij_integrand(part1, part2, z_i, z_j)
        return integrate.dblquad(integrand, self.k_arr[0], self.k_arr[-1], -1, 1)[0]
    
    def get_fisher_ij_integrand(self, part1, part2, z_i, z_j):
        #print('u = ', u, 'k = ', k)
        AP_params_i = {'b': self.cosmo.sigma(z, 8/self.params['h']),
                     'f': self.cosmo.scale_independent_growth_factor_f(z),
                     'a_par': self.c/((1+z)*self.cosmo.Hubble(z)),
                     'a_per': self.cosmo.angular_distance(z)}
        if z_i != z_j:
            AP_params_j = {'b': self.cosmo.sigma(z, 8/self.params['h']),
                         'f': self.cosmo.scale_independent_growth_factor_f(z),
                         'a_par': self.c/((1+z)*self.cosmo.Hubble(z)),
                         'a_per': self.cosmo.angular_distance(z)}
        else:
            AP_params_j = AP_params_i
        def integrand(u,k):
            AP_params_i['dlnPdlnk'] = self.get_dlnPdlnk(u, AP_params_i)
            AP_params_j['dlnPdlnk'] = self.get_dlnPdlnk(u, AP_params_j)
            Pgg_i = self.get_Pgg(u, k, AP_params_i)
            Pgg_j = self.get_Pgg(u, k, AP_params_i)
            Veff = np.sqrt(self.V0*self.n_gal*Pgg_i/(1+self.n_gal*Pgg_i))
            Veff *= np.sqrt(self.V0*self.n_gal*Pgg_j/(1+self.n_gal*Pgg_j))
            ij = part1(u, k, AP_params_i)*part2(u, k, AP_params_j)*Veff
            ij *= np.pi*np.sqrt(1-u**2)/(2*np.pi)**3
            ij *= np.exp(-k**2*self.sigmaz*(1+z)*self.dchidz(z))
            return ij
        return integrand
                
    def get_Pmm(self, z):
        Pk_arr = np.array([self.cosmo.pk(k, z) for k in self.k_arr])
        return interp1d(self.k_arr, Pk_arr, kind='cubic')
    
    def get_Pgg(self, u, k, AP_params):
        Pmm = self.get_Pmm(z)
        b = AP_params['b']
        f = AP_params['f']
        a_par = AP_params['a_par']
        a_per = AP_params['a_per']
        A = a_par/a_per
        vol_corr = 1/(a_par*a_per**2)
        part1 = (k/a_per)*np.sqrt(1+u**2*(A**-2-1))
        part2 = (b+(f*u**2)/(A**2+u**2*(1-A**2)))**2
        return vol_corr*Pmm(k)*part1*part2

    def get_dchidz(self):
        chi = [self.cosmo.comoving_distance(z) for z in self.z_arr]
        dchidz = interp1d(self.z_arr, np.diff(chi, prepend=chi[0]),
                          kind='cubic')
        return dchidz     
    
    def get_dlnPdlnk(self, u, AP_params):
        Pgg_arr = [self.get_Pgg(u, kk, AP_params) for kk in self.k_arr]
        ln_Pgg_arr = np.log(Pgg_arr)
        return interp1d(self.k_arr,
                        np.diff(ln_Pgg_arr, prepend=ln_Pgg_arr[0]),
                        kind='cubic')
        
    def get_dlnPdb(self, u, k, AP_params):
        b = AP_params['b']
        f = AP_params['f']
        return 2/(b+f*u**2)

    def get_dlnPdf(self, u, k, AP_params):
        b = AP_params['b']
        f = AP_params['f']
        return 2*u**2/(b+f*u**2)

    def get_dlnPda_par(self, u, k, AP_params):
        dlnPdlnk = AP_params['dlnPdlnk']
        b = AP_params['b']
        f = AP_params['f']
        dlnPda_par = -1-4*f*u**2*((1-u**2)/(b+f*u**2))
        dlnPda_par += -u**2*dlnPdlnk(k)
        return dlnPda_par

    def get_dlnPda_per(self, u, k, AP_params):
        dlnPdlnk = AP_params['dlnPdlnk']
        b = AP_params['b']
        f = AP_params['f']
        dlnPda_per = -2-4*f*u**2*((1-u**2)/(b+f*u**2))
        dlnPda_per += -(1-u**2)*dlnPdlnk(k)
        return dlnPda_per

# Testing

In [4]:
fish = Fisher(20000, 1/cosmo.pk(0.2, 0))

In [5]:
fisher_mats = fish.get_fisher()

b b
b f
b a_par
b a_per
f f
f a_par
f a_per
a_par a_par
a_par a_per
a_per a_per


In [6]:
fisher_mats

{'bb': 1.5579552049737632e-18,
 'bf': 2.1272475709310798e-19,
 'ba_par': -6.316781569082148e-19,
 'ba_per': -1.1196541385310144e-18,
 'ff': 7.385510744200083e-20,
 'fa_par': -1.2295743251637484e-19,
 'fa_per': -2.0067209386594928e-19,
 'a_para_par': 2.9275913622664433e-19,
 'a_para_per': 5.009215676077834e-19,
 'a_pera_per': 8.85455993633858e-19}